__No te olvides de rellenar esto:__

- Número de grupo: 11
- Nombre de los integrantes del grupo: Mario Alcolea Alcolea, Javier Gómez Moraleda

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

__Fecha de la consulta: 08/04/2021__ 

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [168]:
# Diccionario cuya clave es el identificador y su valor es el nombre
diccionario = {}
# Diccionario cuya clave es el identificador de una clase y su valor es una lista de identificadores de subclases
diccionario_sub = {}
# Diccionario cuya clave es el identificador de una clase y su valor es una lista de identificadores de superclases
diccionario_super = {}

In [169]:
import json

def load(filename):

    # Vaciamos los diccionarios (para evitar que contengan datos de cargas anteriores)
    diccionario.clear()
    diccionario_sub.clear()
    diccionario_super.clear()
    
    with open(filename, encoding="utf8") as f:
        data = json.load(f)

    for i in data:

        # Guardo los identificadores x e y
        x_str = i['x'].replace("http://www.wikidata.org/entity/", "")
        y_str = i['y'].replace("http://www.wikidata.org/entity/", "")

        # Guardo en diccionario con identificadores-nombres
        diccionario[x_str] = i['xLabel']        
        diccionario[y_str] = i['yLabel']

        # Guardo en diccionario de subclases
        if y_str in diccionario_sub:
            diccionario_sub[y_str].append(x_str)
        # Si no existe la clave
        else:
            diccionario_sub[y_str] = [x_str]

        # Guardo en diccionario de superclases
        if x_str in diccionario_super:
            diccionario_super[x_str].append(y_str)
        # Si no existe la clave
        else:
            diccionario_super[x_str] = [y_str]


In [170]:
# Cargamos el fichero de entrada
load("entrada.json")

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [171]:
# Imprime un subárbol de la taxonomía dado un concepto y el nivel máximo de profundidad que nos interesa.
# Los valores level y space siven para llevar el valor actual y la separación lateral (estético) respectivamente.
def print_tree(concept, max_level, level = 0, space = ""):
    
    # Imprimo el espacio, su nivel y el nombre del concepto junto a su identificador entre paréntesis
    print(space + str(level) + " " + diccionario[concept] + " (" + concept + ")")
    
    # Si ese concepto tiene subclases y no me he pasado del nivel desdeado
    if concept in diccionario_sub and level < max_level:
        
        # Para cada subclase, imprimo sus subconceptos de forma recursiva
        for i in diccionario_sub[concept]:
            print_tree(i, max_level, level + 1, space + " ")
            

In [5]:
# Ejemplo jerarquía de voces con 3 niveles de profundidad
print_tree('Q17172850', 3)

0 voz (Q17172850)
 1 bajo (Q27911)
  2 bajo-barítono (Q810480)
   3 lyric bass-baritone (Q54635842)
   3 dramatic bass-baritone (Q54635897)
  2 bajo buffo (Q1002146)
  2 bajo profundo (Q2532487)
  2 bajo cantante (Q3636053)
  2 grave (Q5885030)
  2 octavist (Q7082656)
  2 bajo caractère (Q20638448)
  2 high bass (Q54636007)
  2 dramatic high bass (Q54636036)
  2 serious bass (Q54636068)
  2 heavy acting bass (Q54636271)
  2 Bass bourdon (Q64363543)
 1 barítono (Q31687)
  2 bajo-barítono (Q810480)
   3 lyric bass-baritone (Q54635842)
   3 dramatic bass-baritone (Q54635897)
  2 character baritone (Q1062931)
  2 heldenbaritone (Q1601737)
  2 Barítono buffo (Q5721499)
  2 Barítono ligero (Q5721503)
  2 grave (Q5885030)
  2 Barítono lírico (Q8243255)
  2 Barítono dramático (Q8243257)
  2 baryton-noble (Q19740895)
  2 barítono Martin (Q21478751)
  2 Barítono martín (Q25404193)
  2 barítono verdiano (Q54635681)
  2 acting baritone (Q54635751)
  2 Kavalierbariton (Q54635784)
 1 contralto (Q371

En las bases de conocimiento hay datos erróneos. Hemos encontrado bastantes ejemplos de conceptos que no tienen definida ninguna etiqueta, es decir, no significan realmente nada. Un ejemplo escogido al azar echando un vistazo rápido sobre el diccionario principal que asocia claves con etiquetas sería 'Q13636556'.

In [6]:
# Devuelve la etiqueta de la clave 'Q13636556', que como vemos es la propia clave
diccionario['Q13636556']

'Q13636556'

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [172]:
# Devuelve todos los LCS de dos conceptos dados
def LCS(concept1, concept2):
    
    # Llamada a las funciones que devuelven todos los conceptos más generales de cada uno
    gen_set1 = general_concepts(concept1, [])
    gen_set2 = general_concepts(concept2, [])
    
    # Intersección entre ambos conjuntos
    result_set = gen_set1 & gen_set2
    
    # A partir de aquí, necesito eliminar conceptos más generales de otros conceptos que están en la lista.
    # Es decir, en la lista tengo todos los conceptos comunes, pero si un elemento A de la lista tiene como concepto
    # más general a B, ambos serán conceptos generales de concept1 y concept2 pero sólo me interesa el más específico
    deleted_list = []
    
    # Hago un recorrido para hacer estas eliminaciones
    for i in list(result_set):
        for j in list(result_set):
            
            # Si no es el mismo y j es concepto general de i, lo añado a la lista
            if i != j and j in general_concepts(i, []):
                deleted_list.append(j)

    # Devuelvo la diferencia de los conjuntos
    return result_set - set(deleted_list)

# Función auxiliar que dado un concepto, devuelve una lista de los más generales
def general_concepts(concept, gen_list):
    
    # Añado el propio concepto
    gen_list.append(concept)
    
    # Si tiene algun concepto más general por encima
    if concept in diccionario_super:
        
        # Recorro las superclases y calculo sus conceptos generales
        for i in diccionario_super[concept]:
            
            # Si ya estaban en la lista, significa que ya los he recorrido
            if i not in gen_list:
                
                #Lo añado y calculo sus conceptos generales
                gen_list.append(i)
                general_concepts(i, gen_list)

    # Devuelvo los conceptos como un conjunto
    return set(gen_list)

In [173]:
# mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
# LCS('Q6012297', 'Q54634726') = {'Q186506'}
LCS('Q6012297', 'Q54634726')

{'Q186506'}

In [174]:
# grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
# LCS('Q5885030', 'Q6012300') = {'Q17172850'}
LCS('Q5885030', 'Q6012300')

{'Q17172850'}

In [175]:
# tenor (Q27914)
# LCS('Q27914', 'Q27914') = {'Q27914'}
LCS('Q27914', 'Q27914')

{'Q27914'}

In [176]:
# viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
#LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
LCS('Q15336282', 'Q64166304')

{'Q105738', 'Q1798603', 'Q55724840'}

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [177]:
# Calcula el camino mínimo entre 2 conceptos A y B siendo A más o igual de general que B
def path(concept1, concept2):
    
    # Lista donde almacenaré la ruta mejor
    ruta_mejor = []
    
    # Lista donde almacenaré la ruta que estoy explorando
    ruta_actual = []
    
    # Añado el primer concepto a ambas listas
    ruta_mejor.append(concept1)
    ruta_actual.append(concept1)
    
    # Si los dos conceptos son el mismo, no tengo que hacer nada.
    if concept1 != concept2:
        
        # Si el concepto 2 es subconcepto directo del 1, con añadirlo a la ruta mejor ya tengo el camino mínimo
        if concept2 in diccionario_sub[concept1]:
            ruta_mejor.append(concept2)
        
        # En otro caso, hay uno o más conceptos intermedios
        else:
            
            # Investigo los subconceptos del concepto 1, que al ser distinto del 2, como mínimo tiene un subconcepto
            for i in diccionario_sub[concept1]:
                
                # Añado el subconcepto a la ruta actual
                ruta_actual.append(i)
                
                # Hago una llamada al recorrido en profundidad de ese  concepto
                ruta_mejor = DFS(i, concept2, ruta_mejor, ruta_actual)
                
                # Lo elimino de la ruta actual para investigar el siguiente subconcepto
                ruta_actual.remove(i)

    # Lista para imprimir los nombres ligados a las claves de los conceptos obtenidos en la ruta
    ruta_string = []
    for i in ruta_mejor:
        ruta_string.append(diccionario[i] + " (" + i + ")")
    
    # Devuelvo la mejor ruta 
    return ruta_string

# Función auxiliar que realiza un recorrido en profundidad sobre un concepto concept, hasta llegar a concept2
def DFS(concept, concept2, ruta_mejor, ruta_actual):
    
    # Si el concepto tiene subconceptos (no es hoja)
    if concept in diccionario_sub:
        
        # Para cada subconcepto del concepto
        for i in diccionario_sub[concept]:
            
            # Lo añado a la ruta actual
            ruta_actual.append(i)

            # Si ese subconcepto coincide con concept2, estoy en una ruta solucion
            if(i == concept2):
                
                # Siempre y cuando la solucion sea más corta que la que tenía guardada o bien la solución mejor no era 
                # una solución valida (porque no había llegado a ella), actualizo la mejor solución
                if (len(ruta_mejor) > len(ruta_actual) or concept2 not in ruta_mejor):
                    ruta_mejor = ruta_actual.copy()
                    
            # En otro caso, lanzo un DFS sobre el subconcepto
            else:
                ruta_mejor = DFS(i, concept2, ruta_mejor, ruta_actual)

            # Elimino el subconcepto para explorar los demás
            ruta_actual.remove(i)

    # Devuelvo la mejor ruta
    return ruta_mejor
    

In [178]:
# path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]
path('Q186506', 'Q54634726')

['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']

In [179]:
# path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]
path('Q17172850', 'Q6012300')

['voz (Q17172850)', 'alto (Q6983813)', 'mezzosoprano ligera (Q6012300)']

In [180]:
# path('Q27914', 'Q27914') = [tenor (Q27914)]
path('Q27914', 'Q27914')

['tenor (Q27914)']

In [181]:
# path('Q34379', 'Q55724840') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), 
# lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
path('Q34379', 'Q55724840')

['instrumento musical (Q34379)',
 'cordófono (Q1051772)',
 'composite chordophones (Q19588495)',
 'lutes (Q1808578)',
 'handle lutes (Q30038759)',
 'necked lutes (Q55724833)',
 'necked box lutes (Q55724840)']

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [182]:
def similarity(concept1, concept2):
    
    #Calculamos el LCS que maximiza la similitud
    C_Aux = LCS(concept1, concept2)
    for i in C_Aux:
        C_Aux2 = i
        if i in diccionario_super:
             C = i
        else:
            C = C_Aux2
                
    #Obtenemos el nodo raiz
    for i in diccionario:
        if i not in diccionario_super:
            nodo_raiz = i
    
    caminoRaiz = 0    
    caminoConcept1 = 0
    caminoConcept2 = 0
        
    #camino desde la raíz al LCS de concept1 y concept2
    caminoMinimoRaiz = path(nodo_raiz, C)
    #número de aristas en el camino mínimo
    for i in caminoMinimoRaiz:
        caminoRaiz = caminoRaiz + 1
    caminoRaiz = caminoRaiz - 1    
        
    #camino desde el LCS a concept1
    caminoMinimoConcept1 = path(C, concept1)
    #número de aristas en el camino mínimo
    for i in caminoMinimoConcept1:
        caminoConcept1 = caminoConcept1 + 1
    caminoConcept1 = caminoConcept1 - 1    
        
    #camino desde el LCS a concept2
    caminoMinimoConcept2 = path(C, concept2)
    #número de aristas en el camino mínimo
    for i in caminoMinimoConcept2:
        caminoConcept2 = caminoConcept2 + 1
    caminoConcept2 = caminoConcept2 - 1    
    
    #calculamos la similitud con la fórmula
    similitud = caminoRaiz / (caminoRaiz + caminoConcept1 + caminoConcept2)
    
    #mostramos por pantalla los resultados
    print(similitud)
    print(caminoMinimoRaiz)
    print(caminoMinimoConcept1)
    print(caminoMinimoConcept2)
    

In [183]:
#0.5
#[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
#[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
#[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]
similarity('Q6012297', 'Q54634726')

0.5
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano dramática (Q6012297)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']


In [184]:
#0.6666666666666666
#['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
#['mezzosoprano (Q186506)']
#['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']
similarity('Q186506', 'Q54634726')

0.6666666666666666
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']


In [185]:
#1.0
#['instrumento musical (Q34379)', 'voz (Q17172850)', 'high voice (Q98116969)', 'tenor (Q27914)']
#['tenor (Q27914)']
#['tenor (Q27914)']
similarity('Q27914', 'Q27914')

1.0
['instrumento musical (Q34379)', 'voz (Q17172850)', 'high voice (Q98116969)', 'tenor (Q27914)']
['tenor (Q27914)']
['tenor (Q27914)']


In [186]:
#0.42857142857142855
#['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
#['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)']
#['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)', 'guitarra eléctrica (Q78987)']
similarity('Q76239', 'Q78987')

0.42857142857142855
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [187]:
#piano y guitarra
similarity('Q5994','Q6607')

# La similitud resultante entre el piano y la guitarra es comprensible ya que a pesar de ser ambos instrumentos
# de cuerda pulsada, el piano tiene muchas más superclases que la guitarra, por lo que eso disminuye la similitud.

0.2727272727272727
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)', 'board zithers (Q50829016)', 'true board zithers (Q55724736)', 'true board zithers with resonator (Q55724742)', 'true board zithers with resonator box (Q4951628)', 'piano (Q5994)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']


In [188]:
#guitarra y guitarra eléctrica
similarity('Q6607','Q78987')

# Que la similitud haya dado 0.83 tiene sentido, ya que la guitarra eléctrica es una subclase directa de guitarra 

0.8333333333333334
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']
['guitarra (Q6607)']
['guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


In [189]:
#flauta y trompeta
similarity('Q11405','Q8338')

# Siendo ambos instrumentos instrumentos de viento, la similitud es de 0.4 ya que aunque sea flauta una subclase directa
# de instrumento de viento, la trompeta tiene más superclases, que resta similitud.

0.4
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)']
['instrumento de viento (Q173453)', 'flauta (Q11405)']
['instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


In [190]:
#flauta y piano
similarity('Q11405','Q5994')

# Es normal que la similitud sea 0 porque lo único que tienen en común es que son instrumentos musicales.

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'flauta (Q11405)']
['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)']


In [191]:
#trompeta y guitarra
similarity('Q8338','Q6607')

# Como en el caso anterior, al tener en común únicamente que son instrumentos musicales, sale 0 como resultado.

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']
